# 비화재보 분류 
---

## 1. 데이터 전처리     


In [1]:
import pandas as pd
import numpy as np 
import re

### 각 년도별 데이터 결합 

In [109]:
start_year= 2019
end_year =2021

df_all =pd.DataFrame()
#년도합치기
for year in range(start_year,end_year+1):
    globals()[f'df_{year}']=pd.DataFrame()
#Sheet 합치기
    for page in range(1,10):
        tmp = pd.read_excel(f"{year}년 전체 신고건수_관할 및 재난주소 추가.xls",sheet_name = f'Sheet {page}')['신고내용'].dropna(axis=0).T
        globals()[f'df_{year}']= pd.concat([globals()[f'df_{year}'],tmp], axis = 0)
    df_all= pd.concat([df_all,globals()[f'df_{year}']], axis = 0)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

### 중복 문자열들 제거-set 활용

In [120]:
print(len(df_all))
df_all= pd.DataFrame(set(df_all[0]))
print(df_all.head())
print(len(df_all))

949260
                                                   0
0  [구급상황센터 이관]5살 아기 / 문에 손가락을 찡김 / 손가락 깊이 파임 / 의료...
1                        [구급상황센터 이관]북구 침산동 / 병원안내 / 
2                   지산 2단지 202동 1104호 / 할머니 넘어지짐 위중/
3   매호화성파크드림 101동 508호/남편이 어제부터 상태가 좋지 않다/구토심하고 거동불가
4                                  칠곡경북대 금호 베드 회수 요청
778936


###  신고내용 중 숫자와 영어같은 비화재보 분류와 관련없는 단어들 제거 &전처리 

In [121]:
from konlpy.tag import Okt 
okt = Okt()
print(okt.pos('속보기')) 
#속보기 단어인식을 못함 
#한단어로 인식 X
print(okt.pos('경보기')) 
print(okt.pos('경보')) 
print(okt.pos('속보')) 

[('속', 'Modifier'), ('보기', 'Noun')]
[('경보기', 'Noun')]
[('경보', 'Noun')]
[('속보', 'Noun')]


In [122]:
df_all2 = df_all.copy()
for i in range(len(df_all2)):
    tmp = re.sub(r'[^가-힣+ ]',' ',df_all2.iloc[i,0]) # 한글말고 안나오게
    tmp = re.sub('속보기','속보',tmp)
#속보기 ->속보로 단어 치환    
    df_all2.iloc[i,0] = tmp
print(df_all2.head())

                                                   0
0   구급상황센터 이관  살 아기   문에 손가락을 찡김   손가락 깊이 파임   의료...
1                         구급상황센터 이관 북구 침산동   병원안내   
2                   지산  단지    동     호   할머니 넘어지짐 위중 
3   매호화성파크드림    동    호 남편이 어제부터 상태가 좋지 않다 구토심하고 거동불가
4                                  칠곡경북대 금호 베드 회수 요청


In [123]:
#속보기 단어 제거 확인
print(df_all[0].apply(lambda x: re.search("속보기", x)is not None).sum())
print(df_all2[0].apply(lambda x: re.search("속보기", x)is not None).sum())
print(df_all2[df_all2[0].apply(lambda x: re.search("속보기", x)is not None)])
### 왜 1개 제거가 안됏는지?
# r'[^가      r의 의미

5996
1
                                   0
751563  속보기 앞산        추가 오작동 확인 출동취소


###  문장 -> 형태소 형태로 변환


In [12]:
# 형태소 분석(Noun, Adjective) -명사와 형용사만
# 1자를 배제
# 불용어 제외
sents = []
for i in range(len(df_all2[0])):
    print("\r[tokenizing]  {}  /  {}   {}  %".format(i+1, len(df_all2[0]), round( (i / len(df_all2[0])) * 100 , 1 ) ), end = '    ', flush = False)
    pos_res = okt.pos(df_all2[0].iloc[i])
    
    STOP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    keywords = []
    for word, pos in pos_res:
        if ((pos == "Noun")|(pos == "Adjective"))&(len(word) >= 2)&(word not in STOP_WORDS):
            keywords.append(word)
    sents.append(' '.join(keywords))
print(sents[0:5])

[tokenizing]  778936  /  778936   100.0  %                                                                                                                                                                     

### 경보설비, 속보 설비   //      테스트, 오인주의, 훈련, 점검 문의"  키워드/갯수 확인

In [16]:
df_all2[0].apply(lambda x: re.search("속보", x)is not None).sum()

11027

In [104]:
check = []
for i in range(len(sents)) :
    if '속보' in sents[i]:
            check.append(sents[i])
print(len(check))  


11021


In [18]:
df_all2[0].apply(lambda x: re.search("경보", x)is not None).sum()

8192

In [105]:
check = []
for i in range(len(sents)) :
    if '경보' in sents[i]:
            check.append(sents[i])
print(len(check))  

8108


In [193]:
# 임시저장
df_sents = pd.DataFrame(sents)
df_sents.to_csv("임시저장.csv",encoding= 'euc-kr',index= None)
df_sents= pd.read_csv("임시저장.csv",encoding= 'euc-kr')

### CountVectorizer Vectorizing
랜덤포레스트?

In [159]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
cv_sents=vect.fit_transform(df_sents[0])
print(cv_sents.shape)

(778936, 54861)


### TF-IDF Vectorizing
로지스틱 회귀?

In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(min_df=0.0, analyzer='char', sublinear_tf=True, ngram_range=(1,3), max_features=5000)
vectorizer = TfidfVectorizer()
tfidf_sents =vectorizer.fit_transform(df_sents[0])
print(tfidf_sents.shape)

(778936, 54861)


### Woed2vec(CBOW)
로지스틱 회귀?

In [160]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []
for review in df_sents[0]:
    sentences.append(review.split())

In [162]:
# 파라미터
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3 # 0.001

In [180]:
from gensim.models import word2vec
print("Training model ....")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count, window=context, sample=downsampling)

2022-01-20 14:31:00,645 : INFO : collecting all words and their counts
2022-01-20 14:31:00,646 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-01-20 14:31:00,662 : INFO : PROGRESS: at sentence #10000, processed 91888 words, keeping 7655 word types
2022-01-20 14:31:00,682 : INFO : PROGRESS: at sentence #20000, processed 182808 words, keeping 10892 word types
2022-01-20 14:31:00,700 : INFO : PROGRESS: at sentence #30000, processed 273587 words, keeping 13251 word types
2022-01-20 14:31:00,720 : INFO : PROGRESS: at sentence #40000, processed 364357 words, keeping 15118 word types
2022-01-20 14:31:00,741 : INFO : PROGRESS: at sentence #50000, processed 454946 words, keeping 16795 word types
2022-01-20 14:31:00,768 : INFO : PROGRESS: at sentence #60000, processed 547104 words, keeping 18294 word types
2022-01-20 14:31:00,783 : INFO : PROGRESS: at sentence #70000, processed 639852 words, keeping 19613 word types
2022-01-20 14:31:00,800 : INFO : PROGRESS: at se

Training model ....


2022-01-20 14:31:00,830 : INFO : PROGRESS: at sentence #100000, processed 910752 words, keeping 22997 word types
2022-01-20 14:31:00,846 : INFO : PROGRESS: at sentence #110000, processed 1001710 words, keeping 24024 word types
2022-01-20 14:31:00,865 : INFO : PROGRESS: at sentence #120000, processed 1092306 words, keeping 24981 word types
2022-01-20 14:31:00,879 : INFO : PROGRESS: at sentence #130000, processed 1182461 words, keeping 25860 word types
2022-01-20 14:31:00,892 : INFO : PROGRESS: at sentence #140000, processed 1272269 words, keeping 26744 word types
2022-01-20 14:31:00,908 : INFO : PROGRESS: at sentence #150000, processed 1363019 words, keeping 27581 word types
2022-01-20 14:31:00,930 : INFO : PROGRESS: at sentence #160000, processed 1453211 words, keeping 28367 word types
2022-01-20 14:31:00,943 : INFO : PROGRESS: at sentence #170000, processed 1544012 words, keeping 29161 word types
2022-01-20 14:31:00,959 : INFO : PROGRESS: at sentence #180000, processed 1636128 words, 

In [176]:
def get_features(words,model, num_features):
    feature_vector = np.zeros((num_features), dtype=np.float32)
    num_words = 0
    index2word_set = set(model.wv.index2word)
    for w in words:
        if w in index2word_set:
            num_words +=1
            feature_vector = np.add(feature_vector, model[w])
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [177]:
def get_dataset(sentence, model, num_features):
    dataset = list()
    for s in sentence:
        dataset.append(get_features(s, model, num_features))
    FeatureVecs = np.stack(dataset)
    return FeatureVecs

In [178]:
train_data_vecs = get_dataset(sentences, model, num_features)
print(train_data_vecs.shape)

C:\Users\user\AppData\Local\Temp/ipykernel_12864/1152472743.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feature_vector = np.add(feature_vector, model[w])
C:\Users\user\AppData\Local\Temp/ipykernel_12864/1152472743.py:10: RuntimeWarning: invalid value encountered in true_divide
  feature_vector = np.divide(feature_vector, num_words)
